In [1]:
!pip install --upgrade --no-cache-dir gdown

In [2]:
!pip install -U gensim

In [3]:
import gdown
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt 
%matplotlib inline 
import string
from tqdm import tqdm_notebook as tqdm
import re
from scipy import spatial
import heapq
import timeit

#NLTK
import nltk
from nltk.corpus import stopwords
#nltk.download("english")
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
pd.set_option('display.max_columns', None)

**Задаа 0: Загрузим и предобработаем данные**

In [5]:
!gdown 1Co7iQWJhJdqNFTlChd-xT1b88VAzuX4f
!gdown 14OAYTUIw9BoCCSTFIiYzgPq-1XyP0dhQ

Downloading...
From: https://drive.google.com/uc?id=1Co7iQWJhJdqNFTlChd-xT1b88VAzuX4f
To: /content/movies_metadata.csv
100% 34.4M/34.4M [00:00<00:00, 143MB/s]
Downloading...
From: https://drive.google.com/uc?id=14OAYTUIw9BoCCSTFIiYzgPq-1XyP0dhQ
To: /content/ratings.csv
100% 710M/710M [00:04<00:00, 151MB/s]


**movies_metadata.csv:** The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

**ratings.csv:** Ratings of the Full Dataset.

In [6]:
movies_metadata = pd.read_csv('movies_metadata.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
movies_metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [8]:
# удалим ошибочные значения

movies_metadata.drop(movies_metadata[movies_metadata['id'] == '1997-08-20'].index, inplace=True)

In [9]:
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '2012-09-29'].index, inplace=True)

In [10]:
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '2014-01-01'].index, inplace=True)

In [11]:
# оставим только колонки, которые мы собираемся использовать

movies_metadata = movies_metadata[['id', 'original_title', 'overview']]

In [12]:

# заполняем пустые значения

movies_metadata = movies_metadata.fillna('')

In [13]:
# удалим дубли, потому что они есть и очень мешаются

movies_metadata = movies_metadata.drop_duplicates().reset_index(drop=True)

In [14]:

movies_metadata = movies_metadata.rename(columns={'id':'movieId'})

**1 Получим случайную матрицу эмбедингов для каждого фильма**

In [15]:
def preprocessing(data, lemmatizer = WordNetLemmatizer(), 
                  stop_words = stopwords.words('english')):
    
    '''
       на вход: 
       - data - датафрейм с описанием и названием фильмов
       - lemmatizer - класс для лемматизации текстов
       - stop_words - стоп-слова
      
       на выходе: 
       - список с предобработанными текстовыми данными
    '''
    
    texts = data.values.tolist()
    new_text = []

    for i in tqdm(range(len(texts))):
        text = texts[i]

        # приводим к нижнему регистру и дропаем стоп-слова
        punct_free = re.sub('[^a-zA-Z]', ' ', str(text).lower())
        punct_free = re.sub(r'\s+', ' ', punct_free)
        stop_free = " ".join([i for i in punct_free.split() if i not in stop_words])
        lemma_text = ' '.join([lemmatizer.lemmatize(word) for word in stop_free.split(' ')])
        new_text.append(lemma_text)
    
    return new_text

In [16]:
nltk.download("stopwords")

text_overview = preprocessing(movies_metadata["overview"])
text_title = preprocessing(movies_metadata["original_title"])

movies_metadata["overview"] = text_overview
movies_metadata["original_title"] = text_title 

# закидываем предобработанные описания и названия фильмов в общий список

text_prepr = text_title + text_overview

text_prepr = [i.split() for i in text_prepr]
text_overview = [i.split() for i in text_overview]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/45433 [00:00<?, ?it/s]

  0%|          | 0/45433 [00:00<?, ?it/s]

In [18]:
# word2vec

model = Word2Vec(text_prepr, 
                 min_count=2)

model.build_vocab(text_prepr)
words = model.wv.index_to_key
vocab_size = len(words)

print("Vocab size =", vocab_size)

Vocab size = 42212


In [30]:
def embedding_matrix(text, model):
    
    """
       На вход: словарь tokenizer.word_index 
       модель word2vec
        
       На выходе: матрица эмбедингов текста
    """
    
    embedding_matrix = np.zeros((id_film, 100))

    for sentence in text:
        for word in sentence:
            if word in model:
                embedding_matrix[text.index(sentence)] = model[word]

    return embedding_matrix

In [31]:
id_film = movies_metadata['movieId'].nunique()

matrix_emb = embedding_matrix(text_overview, model.wv)

matrix_emb = pd.concat([movies_metadata[['movieId']],
                        pd.DataFrame(matrix_emb)], axis=1)

In [33]:
matrix_emb.tail(2)

,movieId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
45431,227506,-0.123376,0.260597,-0.010412,-0.028873,-0.153527,-0.065952,0.067863,0.186642,-0.004108,0.170699,-0.017260,0.121448,-0.014860,-0.044359,0.183356,-0.020156,0.064094,-0.130689,0.004036,-0.110691,-0.090186,0.040070,-0.065794,0.022758,-0.066866,0.046950,-0.095876,-0.045222,-0.081019,-0.010852,0.211779,0.121557,0.109212,-0.029708,-0.170051,0.125463,-0.030371,0.170310,0.070700,-0.153936,0.108811,0.065799,0.029390,-0.131893,-0.070429,0.102200,-0.085648,0.169564,-0.042904,0.212477,-0.123197,0.019601,0.003969,0.073543,0.049847,-0.042663,-0.058538,-0.003372,0.085090,-0.023715,0.257914,0.037157,0.003701,-0.004607,0.014456,-0.010448,0.187024,0.002277,0.012469,0.112906,0.001139,-0.046752,-0.072266,0.013637,0.016278,0.127938,-0.035151,-0.007569,0.084395,0.216894,0.185072,-0.023627,0.068087,-0.071371,0.026015,0.011475,-0.112707,0.193898,0.178344,-0.077624,-0.036096,-0.054316,-0.020588,0.077510,0.187728,0.035347,-0.002895,-0.078833,0.000258,-0.125582
45432,461257,0.486193,0.479562,0.267347,0.346940,0.067085,-0.491323,0.768858,0.707046,0.049762,0.210596,0.222048,0.467426,0.515358,0.363856,0.209615,0.103222,0.433976,-0.341717,-0.136924,-0.578998,-0.497350,0.497621,0.044261,-0.403033,-0.278592,0.484296,-0.370640,0.428276,-0.297632,-0.303507,0.499146,0.667763,-0.053328,-0.063930,-0.664887,0.839213,-0.607704,0.245421,0.376301,-1.338401,0.753777,0.232634,-0.291558,0.063085,-0.127980,0.653523,-0.320170,-0.015490,0.071579,0.565610,-0.390103,0.219444,-0.202685,-0.246305,0.315113,0.292598,-1.115871,0.249668,0.112576,-0.437800,1.256567,0.419998,-0.043271,-0.195507,-0.109614,0.186824,0.075325,0.052017,-0.727937,0.425456,0.281324,-0.264098,0.461453,0.428777,0.064534,0.421308,-0.271360,-0.077856,-0.471198,0.606468,0.345678,0.212304,-0.369765,0.339600,-0.407744,-0.080662,-0.319869,0.463185,0.638922,-0.362853,-0.070660,-0.155283,-0.834655,0.644207,0.729154,0.024312,-0.138874,-0.392889,0.194051,-1.146949


In [ ]:
# из исходных данных берем айдишник и добавляю 300 случайных колонок

#matrix_emb = pd.concat([movies_metadata[['id']], 
#                        pd.DataFrame(np.random.random((movies_metadata.shape[0], 300)))], 
#                       axis=1)

In [34]:
# меняем тип данных колонки, чтобы проще было джойниться с другой таблицей

matrix_emb['movieId'] = matrix_emb['movieId'].astype(int)

**2 Теперь напишем функцию train_test_split по юзеру по матрице рейтингов (юзер х фильмы = рейтинг)**

**Подаем на вход функции юзера и получаем два списка фильмов: на тренировочном будем учить, а фильмы из тестового набора искать в предсказаниях**


In [35]:
ratings = pd.read_csv('ratings.csv')

In [36]:
ratings.sample(5)

,userId,movieId,rating,timestamp
7037362,72556,44828,5.0,1460671118
13291050,138174,232,4.0,835969619
18251930,189349,2081,0.5,1137820814
16700871,173718,93840,5.0,1355004225
10619467,109585,2611,3.0,942960377


In [37]:
# оказвается, только к 7565 фильмам из movies_metadata есть оценки в ratings

common_ids = set(matrix_emb.movieId).intersection(set(ratings.movieId))

len(common_ids)

7565

In [38]:
# выберем только пересекающиеся фильмы

#matrix_emb = matrix_emb[matrix_emb.id.isin(common_ids)]

In [39]:
#matrix_emb.shape

In [40]:
ratings = ratings[ratings.movieId.isin(common_ids)]

In [41]:
len(ratings.movieId.unique())

7565

In [42]:
def train_test_split_films(dataframe, user_list, size=0.7):

  '''
     на вход подается:
     - dataframe - название датафрейма с рейтингами
     - user_list - список пользователей
     - size - пропорция разделения

     на выходе для всех пользователей получаем один словарь, где
     ключ - id зрителя
     список 1 - айдишники train фильмов
     список 2 - оценки train фильмов
     список 3 - айдишники test фильмов
     список 4 - оценки test фильмов
  '''

  user_tts_data_dict = {}

  # итерируемся по всем пользователям
  for user in user_list:

    # выбираю данные по нужному юзеру и перемешиваю
    temp_df = dataframe[dataframe['userId'] == user] \
              .sample(dataframe[dataframe['userId'] == user].shape[0]) \
              [['movieId', 'rating']]

    # отфильтровываем пользователей с количеством просмотров < 10
    if temp_df.shape[0] < 10:
      continue

    # вычисляю длину тренировочного набора для нужного юзера
    shape = int(temp_df.shape[0]*size)

    # сразу записываем в словарь четыре вектора
    user_tts_data_dict[user] = [temp_df.iloc[:shape]['movieId'].tolist(),
                                temp_df.iloc[:shape]['rating'].tolist(),
                                temp_df.iloc[shape:]['movieId'].tolist(),
                                temp_df.iloc[shape:]['rating'].tolist()]

  return user_tts_data_dict

In [43]:
# вторым параметром на вход можно подать любой список пользователей

splitted_data = train_test_split_films(ratings, ratings['userId'].unique()[:10], 0.7)

**3 Для каждого юзера по train-фильмам находим средний вектор эмбединг через mean**

**Надо дополнительно домножить эмбединг-вектор на оценку фильма**

In [44]:
def get_avg_weightened_vec(dict_name, movies_emb_df):

  '''
     на вход подается:
     - dict_name - название словаря с данными после сплита 
     - movies_emb_df - название эмбединг-датасета с фильмами

     далее мы находим все эмбединг-вектора для train фильмов, домножаем
     каждый на оценку от зрителя и находим средний вектор для каждого зрителя

     на выходе:
     - словарь с ключами - айди пользователей и значениями - средним взвешенным 
     вектором по просмотренным фильмам в train-наборе по каждому пользователю
  '''

  user_vec_data_dict = {}

  # итерируемся по всем пользователям
  for key in dict_name.keys():

    # находим все вектора фильмов из train-выборки в эмбединг-матрице
    # и джойним с их рейтингами от выбранного пользователя
    temp_df = movies_emb_df \
              .loc[movies_emb_df['movieId'].isin(dict_name[key][0])] \
              .merge(pd.DataFrame(dict_name[key][0], dict_name[key][1], 
                                  columns=['movieId'])
                                  .reset_index()
                                  .rename(columns={'index':'rating'}), on='movieId')

    # домножаем эмбединг-вектора на веса-оценки и вычисляем средний вектор
    avg_weightened_vec = list(temp_df \
                              .drop(['movieId'], axis=1)
                              .mul(temp_df['rating'], axis=0)
                              .drop(['rating'], axis=1)
                              .sum()
                              .mul(1 / temp_df['rating'].sum(), axis=0))
    
    # все вектора записываем в словарь под ключ-айди пользователя
    user_vec_data_dict[key] = avg_weightened_vec

  return user_vec_data_dict

In [45]:
# получим матрицу пользователей

res_vec_dict = get_avg_weightened_vec(splitted_data, matrix_emb)

In [46]:
# преобразуем к табличному виду

res_vec_df = pd.DataFrame.from_dict(res_vec_dict, orient='index').reset_index().rename(columns={'index':'movieId'})

In [47]:
res_vec_df.head(2)

,movieId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,1,-0.116851,0.341553,0.023092,-0.098854,-0.094760,-0.230436,-0.091827,0.292292,0.113156,0.126396,0.155214,0.218243,-0.153593,0.068783,0.297086,-0.053724,0.203249,-0.193399,0.023484,-0.315316,0.158826,-0.012321,-0.110607,-0.025465,-0.149758,0.227684,-0.303754,-0.108965,-0.192826,0.087905,0.273165,0.359180,0.081503,-0.126306,-0.156215,-0.021504,-0.269597,0.064820,-0.121482,-0.181614,0.141695,0.084315,0.262776,-0.292554,-0.165232,-0.013255,0.066813,0.083863,0.013515,0.353883,-0.274992,0.087812,-0.082123,0.159872,-0.132381,-0.041578,-0.056033,0.079976,0.199848,-0.040406,0.417916,-0.071858,-0.000472,-0.008932,0.093920,0.036336,0.283933,0.048163,0.014377,0.235621,-0.025878,-0.154037,-0.211345,-0.102290,0.076129,0.116888,-0.053195,-0.053274,-0.033562,0.228288,0.138264,-0.029514,-0.019953,-0.108138,-0.014574,0.115980,-0.047882,0.288394,0.169949,-0.120965,0.180842,-0.162271,-0.12035,-0.049169,0.203238,0.034542,0.118994,-0.144365,-0.055124,0.103820
1,2,-0.052694,0.725947,0.277627,0.261873,0.091904,-0.617426,0.131195,0.523576,0.031851,0.151546,0.039124,0.390129,-0.119891,0.306863,0.258692,0.014233,0.374146,-0.442065,-0.252823,-0.815155,-0.132351,0.397343,0.000202,-0.326630,-0.327048,0.397380,-0.366206,0.167084,-0.171629,-0.067907,0.389755,0.699865,0.165614,0.102288,-0.488982,0.187472,-0.290953,0.277909,-0.022974,-0.788898,0.459055,-0.067774,-0.112731,-0.089345,-0.093962,0.243987,-0.573943,0.084348,0.255860,0.559790,-0.270047,0.179141,-0.079312,0.198988,0.211578,0.140934,-0.281179,0.086723,0.037751,-0.241149,0.795756,0.114365,-0.121724,-0.237482,-0.070601,0.337608,0.309482,0.006332,-0.433834,0.709395,0.201768,-0.118427,-0.068860,-0.191304,0.174084,0.119199,0.028845,-0.234394,-0.206331,0.082711,0.042736,0.142164,-0.369696,0.184447,-0.133603,0.010921,-0.105712,0.518010,0.532323,0.054913,0.088844,0.159873,-0.29818,0.368201,0.564052,-0.082268,-0.003280,-0.157058,-0.190652,-0.252733


**Задача 4: Найдем топ близких фильмов, которые пользователь еще не смотрел (косинусное расстояние)**

In [48]:
def get_closest_films(usr_avg_vec_df, user_id, films_emb_mtrx, user_data_dict, top_n=10):

  '''
     на вход подается:
     - usr_avg_vec_df - матрица с train-предпочтениями пользователя
     - user_id - айди пользователя, для которого мы хотим получить рекомендации
     - films_emb_mtrx - эмбединг матрица фильмов
     - splitted_data_dict - словарь с train_test data для всех пользователей
     - top_n - количество рекомендуемых фильмов

     на выходе:
     - список наиболее подходящих непросмотренных фильмов
  '''

  scores = {}

  # достаем средний вектор предпочтений пользователя
  user_avg_vec = usr_avg_vec_df[usr_avg_vec_df['movieId'] == user_id] \
                                .drop('movieId', axis=1) \
                                .values \
                                .flatten() \
                                .tolist()

  # итерируемся по списку фильмов и находим косинусное расстояние между
  # средним вектором пользователя и каждым фильмом
  for film in films_emb_mtrx['movieId']:
    
    film_vec = films_emb_mtrx[films_emb_mtrx['movieId'] == film] \
                              .drop('movieId', axis=1) \
                              .values \
                              .flatten() \
                              .tolist()

    # промежуточные результаты записываем в словарь
    scores[film] = 1 - spatial.distance.cosine(user_avg_vec, film_vec)

  # удаляем из словаря фильмы, которые пользователь уже смотрел (в train-наборе)
  for viewed_film in user_data_dict[user_id][0]:
    del scores[viewed_film]

  # возвращаем топ фильмов в порядке убывания близости к вектору пользователя
  # записываем их в пятый список под айди пользователя в словаре с train-test-data
  user_data_dict[user_id].append(heapq.nlargest(top_n, scores, key=scores.get))

In [49]:
# получаем предсказания для пользователей

for i, user in enumerate(splitted_data.keys()):
  print('Running:', i + 1, '/', len(splitted_data))
  get_closest_films(res_vec_df, user, matrix_emb, splitted_data)

Running: 1 / 8


/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Running: 2 / 8
Running: 3 / 8
Running: 4 / 8
Running: 5 / 8
Running: 6 / 8
Running: 7 / 8
Running: 8 / 8


**Задача 5: Проверить результаты по метрикам**

In [50]:
def mean_precision(user_data_dict):

  '''
     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     средняя метрика precision@k по всем пользователям
  '''

  sum_precision = 0

  # вычисляем метрику по каждому пользователю
  for user in user_data_dict.keys():
    TP = len(set(user_data_dict[user][4]).intersection(set(user_data_dict[user][2])))
    k = len(user_data_dict[user][4])
    
    sum_precision += TP / k

  # находим среднее по пользователям
  return sum_precision / len(user_data_dict)

In [51]:
def mean_recall(user_data_dict):

  '''
     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     средняя метрика recall@k по всем пользователям
  '''

  sum_recall = 0

  # вычисляем метрику по каждому пользователю
  for user in user_data_dict.keys():
    TP = len(set(user_data_dict[user][4]).intersection(set(user_data_dict[user][2])))
    TP_FN = len(user_data_dict[user][2])
    
    sum_recall += TP / TP_FN

  # находим среднее по пользователям
  return sum_recall / len(user_data_dict)

In [52]:
def MRR(user_data_dict):

  '''
     RR - обратный ранг, 1 / индеск первого вхождения рекомендации в тест

     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     метрика MRR@k по всем пользователям
  '''

  sum_rr = 0

  # находим порядковый номер первого вхождения правильной рекомендации в тест
  for user in user_data_dict.keys():
    rank = [1 + splitted_data[1][4].index(x) for x in \
            set(splitted_data[1][4]).intersection(set(splitted_data[1][2]))]
    
    # находим обратный ранк, если попадания нет, то зануляем
    if len(rank) == 0:
      rr = 0
    else:
      rr = 1 / min(rank)

    sum_rr += rr

  # находим среднее по пользователям
  return sum_rr / len(user_data_dict)

In [53]:
def mean_average_precision_at_k(user_data_dict):

  '''
     Mean Average Precision - средняя точность по пользователям

     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     метрика MAP@k по всем пользователям
  '''

  sum_ap = 0

  # итерируемся по пользователям
  for user in user_data_dict.keys():

    num_hits = 0
    score = 0

    for i, p in enumerate(user_data_dict[user][4]):

      if p in user_data_dict[user][2] and p not in user_data_dict[user][4][:i]:
        num_hits += 1
        score += num_hits / (i + 1)

    sum_ap += score / min(len(user_data_dict[user][2]), len(user_data_dict[user][4]))

  return sum_ap / len(user_data_dict)

In [54]:
print('Mean Precision: ', mean_precision(splitted_data))
print('Mean Recall: ', mean_recall(splitted_data))
print('Mean Reciprocal Rank:', MRR(splitted_data))
print('Mean Average Precision at K:', mean_average_precision_at_k(splitted_data))

Mean Precision:  0.0
Mean Recall:  0.0
Mean Reciprocal Rank: 0.0
Mean Average Precision at K: 0.0
